# $\color{black}{\text{Kapittel 1 - Følger og rekker}}$

# $\color{red}{\text{Kapittel 2 - Integralregning}}$

In [ ]:
# -*- coding: utf-8 -*-
import sympy as sp
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output, Math, Markdown

# --- SYMBOLSK OPPSETT ---
# Definer symboler. 'oo' er SymPys uendelighet.
x, y = sp.symbols('x y')
oo = sp.oo

# --- HJELPEFUNKSJONER ---

def parse_expression(expr_str):
    """Trygt parser en streng til et SymPy-uttrykk."""
    try:
        # Erstatt standard potensoperator med Pythons versjon
        safe_expr_str = expr_str.replace('^', '**')
        # Sympify konverterer strengen til et matematisk uttrykk
        return sp.sympify(safe_expr_str)
    except (sp.SympifyError, SyntaxError) as e:
        raise ValueError(f"Ugyldig funksjonsuttrykk: '{expr_str}'. Feil: {e}")

def create_numpy_function(expr):
    """Konverterer et SymPy-uttrykk til en rask NumPy-funksjon for plotting."""
    # lambdify er den anbefalte måten å gjøre dette på. Mye tryggere enn eval().
    return sp.lambdify(x, expr, modules=['numpy'])

# --- KJERNEFUNKSJONER (LOGIKK FOR HVER OPERASJON) ---

def handle_derivation(f_expr):
    derivative = sp.diff(f_expr, x)
    display(Math(f"f'(x) = \\frac{{d}}{{dx}} \\left( {sp.latex(f_expr)} \\right) = {sp.latex(derivative)}"))

def handle_indefinite_integral(f_expr):
    antiderivative = sp.integrate(f_expr, x)
    # Legg til "+ C" for å være fullstendig
    display(Math(f"\\int \\left( {sp.latex(f_expr)} \\right) \\,dx = {sp.latex(antiderivative)} + C"))

def handle_definite_integral(f_expr, a, b):
    antiderivative = sp.integrate(f_expr, x)
    display(Math(f"\\text{{1. Finn den antideriverte: }} F(x) = \\int \\left( {sp.latex(f_expr)} \\right) \\,dx = {sp.latex(antiderivative)}"))
    
    result = sp.integrate(f_expr, (x, a, b))
    # Vis utregningen med innsatte grenser
    eval_b = antiderivative.subs(x, b)
    eval_a = antiderivative.subs(x, a)
    display(Math(f"\\text{{2. Evaluer grensene: }} F(b) - F(a) = \\left( {sp.latex(eval_b)} \\right) - \\left( {sp.latex(eval_a)} \\right)"))

    # Vis det endelige svaret, både eksakt og som desimaltall
    display(Math(f"\\text{{Resultat: }} \\int_{{{a}}}^{{{b}}} \\left( {sp.latex(f_expr)} \\right) \\,dx = {sp.latex(result)} \\approx {result.evalf(6)}"))
    plot_functions('Bestemt integral', f_expr=f_expr, a=float(a), b=float(b))

def handle_area_between_graphs(f_expr, g_expr, a, b, find_intersections):
    if find_intersections:
        try:
            # Løs f(x) = g(x) for x, kun reelle løsninger
            intersections = sp.solve(f_expr - g_expr, x, domain=sp.S.Reals)
            intersections.sort()
            if len(intersections) >= 2:
                a, b = intersections[0], intersections[-1]
                a_input.value = float(a)
                b_input.value = float(b)
                display(Markdown(f"**Skjæringspunkter funnet og brukt som grenser:** `a = {a}` og `b = {b}`"))
            else:
                display(Markdown("**Fant færre enn 2 reelle skjæringspunkter. Bruker manuelle grenser.**"))
        except NotImplementedError:
             display(Markdown("**Kunne ikke løse for skjæringspunkter analytisk. Bruk manuelle grenser.**"))

    # Integrer absoluttverdien av differansen for å sikre positivt areal
    integrand = sp.Abs(f_expr - g_expr)
    area = sp.integrate(integrand, (x, a, b))
    
    display(Math(f"\\text{{Areal }} A = \\int_{{{a}}}^{{{b}}} |f(x) - g(x)| \\,dx = \\int_{{{a}}}^{{{b}}} \\left| {sp.latex(f_expr)} - \\left( {sp.latex(g_expr)} \\right) \\right| \\,dx"))
    display(Math(f"A = {sp.latex(area)} \\approx {area.evalf(6)}"))
    plot_functions('Areal mellom grafer', f_expr=f_expr, g_expr=g_expr, a=float(a), b=float(b))

def handle_volume_revolution(f_expr, g_expr, a, b, axis='x'):
    if axis == 'x':
        # Volum ved rotasjon om x-aksen (skive-/vaskemetoden)
        integrand = sp.pi * (f_expr**2 - g_expr**2)
        display(Markdown("### Volum av omdreiningslegeme om x-aksen (Skivemetoden)"))
        display(Math(f"V = \\pi \\int_{{{a}}}^{{{b}}} \\left( (f(x))^2 - (g(x))^2 \\right) \\,dx"))
        display(Math(f"V = \\pi \\int_{{{a}}}^{{{b}}} \\left( \\left({sp.latex(f_expr)}\\right)^2 - \\left({sp.latex(g_expr)}\\right)^2 \\right) \\,dx"))
    else: # axis == 'y'
        # Volum ved rotasjon om y-aksen (sylinderskallmetoden)
        integrand = 2 * sp.pi * x * (f_expr - g_expr)
        display(Markdown("### Volum av omdreiningslegeme om y-aksen (Sylinderskallmetoden)"))
        display(Math(f"V = 2\\pi \\int_{{{a}}}^{{{b}}} x \\left( f(x) - g(x) \\right) \\,dx"))
        display(Math(f"V = 2\\pi \\int_{{{a}}}^{{{b}}} x \\left( \\left({sp.latex(f_expr)}\\right) - \\left({sp.latex(g_expr)}\\right) \\right) \\,dx"))

    volume = sp.integrate(integrand, (x, a, b))
    display(Math(f"V = {sp.latex(volume)} \\approx {volume.evalf(6)}"))
    plot_functions('Volum', f_expr=f_expr, g_expr=g_expr, a=float(a), b=float(b), title_extra=f"om {axis}-aksen")

def handle_numerical_integration(f_str, a, b):
    # Bruk SciPy.quad for rask og nøyaktig numerisk integrasjon
    f_numpy = lambda x_val: eval(f_str, {"x": x_val, "np": np})
    result, error = integrate.quad(f_numpy, a, b)
    display(Markdown(f"**Resultat:** `{result}`"))
    display(Markdown(f"**Estimert feilmargin:** `{error}`"))
    
    # For plotting må vi parse uttrykket til SymPy først
    f_expr = parse_expression(f_str)
    plot_functions('Numerisk integral', f_expr=f_expr, a=a, b=b)

# --- PLOTTEFUNKSJON ---

def plot_functions(op_name, f_expr, g_expr=None, a=0, b=1, title_extra=""):
    # Lag numeriske funksjoner for plotting
    f_numpy = create_numpy_function(f_expr)
    
    # Lag et tett rutenett av x-verdier for en jevn graf
    x_vals = np.linspace(a, b, 500)
    y_vals_f = f_numpy(x_vals)

    plt.figure(figsize=(10, 6))
    plt.plot(x_vals, y_vals_f, label=f'f(x) = {f_expr}', color='blue')
    
    plot_title = f"Visualisering av {op_name} {title_extra}"
    
    if g_expr is not None:
        g_numpy = create_numpy_function(g_expr)
        y_vals_g = g_numpy(x_vals)
        plt.plot(x_vals, y_vals_g, label=f'g(x) = {g_expr}', color='green')
        
        # Fyll området mellom grafene
        plt.fill_between(x_vals, y_vals_f, y_vals_g, where=(y_vals_f > y_vals_g), 
                         interpolate=True, color='purple', alpha=0.3, label='Areal (f > g)')
        plt.fill_between(x_vals, y_vals_f, y_vals_g, where=(y_vals_g >= y_vals_f), 
                         interpolate=True, color='orange', alpha=0.3, label='Areal (g >= f)')

        # Finn og plott skjæringspunkter i intervallet
        try:
            intersections = sp.solve(f_expr - g_expr, x, domain=sp.S.Reals)
            real_intersections_in_range = [i for i in intersections if a <= i <= b]
            if real_intersections_in_range:
                y_intersections = f_numpy(np.array(real_intersections_in_range, dtype=float))
                plt.scatter(real_intersections_in_range, y_intersections, color='red', zorder=5, label='Skjæringspunkter')
        except NotImplementedError:
            pass # Ignorer om SymPy ikke kan løse ligningen

    elif op_name in ['Bestemt integral', 'Numerisk integral']:
        plt.fill_between(x_vals, y_vals_f, alpha=0.3, color='orange', label='Integrasjonsområde')

    plt.axhline(0, color='black', linewidth=0.5)
    plt.axvline(0, color='black', linewidth=0.5)
    plt.title(plot_title)
    plt.xlabel("x")
    plt.ylabel("y")
    plt.legend()
    plt.grid(True)
    plt.ylim(min(y_vals_f.min(), 0) - 1, y_vals_f.max() + 1) # Juster y-aksen
    plt.xlim(a - 0.1*(b-a), b + 0.1*(b-a)) # Litt pusterom på x-aksen
    plt.show()

# --- OPPSETT AV WIDGETS (BRUKERGRENSESNITT) ---

# Beskrivelse av verktøyet
display(Markdown("# R2 Kalkulus-verktøy"))
display(Markdown("Dette verktøyet utfører symbolske og numeriske beregninger for derivasjon og integrasjon. "
                 "Bruk standard Python-syntaks for funksjoner, f.eks. `x**2` for $x^2$, `sp.sin(x)` for $\\sin(x)$, `sp.exp(x)` for $e^x$. "
                 "For uendelig, skriv `oo` (to små o-er)."))

# Hovedmeny
operation = widgets.Dropdown(
    options=[
        'Derivasjon',
        'Ubestemt integral',
        'Bestemt integral',
        'Areal mellom grafer',
        'Volum om x-aksen',
        'Volum om y-aksen',
        'Numerisk integral'
    ],
    description='Velg operasjon:',
    style={'description_width': 'initial'}
)

# Inndatafelt
function_input = widgets.Text(value='sin(x)', description='f(x):', layout={'width': '400px'})
function_g_input = widgets.Text(value='cos(x)', description='g(x):', layout={'width': '400px'})
a_input = widgets.Text(value='0', description='a (nedre grense):')
b_input = widgets.Text(value='pi/2', description='b (øvre grense):')
find_intersections_checkbox = widgets.Checkbox(value=False, description='Finn grenser automatisk')
run_button = widgets.Button(description="Kjør beregning", button_style='success')

# Område for output
output_area = widgets.Output()

# Ordne widgets i bokser for bedre layout
input_widgets = widgets.VBox([
    operation,
    function_input,
    function_g_input,
    widgets.HBox([a_input, b_input]),
    find_intersections_checkbox
])

# --- LOGIKK FOR BRUKERGRENSESNITT ---

def update_ui(*args):
    """Viser/skjuler widgets basert på valgt operasjon."""
    op = operation.value
    function_g_input.layout.display = 'none'
    a_input.layout.display = 'none'
    b_input.layout.display = 'none'
    find_intersections_checkbox.layout.display = 'none'

    if op == 'Areal mellom grafer':
        function_g_input.layout.display = 'flex'
        a_input.layout.display = 'flex'
        b_input.layout.display = 'flex'
        find_intersections_checkbox.layout.display = 'flex'
    elif op in ['Bestemt integral', 'Numerisk integral']:
        a_input.layout.display = 'flex'
        b_input.layout.display = 'flex'
    elif op in ['Volum om x-aksen', 'Volum om y-aksen']:
        function_g_input.description = "g(x) (nedre kurve):"
        function_g_input.value = "0" # Standard er rotasjon av areal under én graf
        function_g_input.layout.display = 'flex'
        a_input.layout.display = 'flex'
        b_input.layout.display = 'flex'

operation.observe(update_ui, names='value')

def run_calculation_on_click(b):
    """Hovedfunksjon som kalles når knappen trykkes."""
    with output_area:
        clear_output(wait=True)
        try:
            op = operation.value
            f_str = function_input.value
            f_expr = parse_expression(f_str)

            # Hent grenser kun om de trengs
            if op not in ['Derivasjon', 'Ubestemt integral']:
                a = parse_expression(a_input.value)
                b = parse_expression(b_input.value)
            
            # Hent g(x) kun om det trengs
            if op in ['Areal mellom grafer', 'Volum om x-aksen', 'Volum om y-aksen']:
                g_expr = parse_expression(function_g_input.value)

            # Kall riktig logikkfunksjon basert på valg
            if op == 'Derivasjon':
                handle_derivation(f_expr)
            elif op == 'Ubestemt integral':
                handle_indefinite_integral(f_expr)
            elif op == 'Bestemt integral':
                handle_definite_integral(f_expr, a, b)
            elif op == 'Areal mellom grafer':
                handle_area_between_graphs(f_expr, g_expr, a, b, find_intersections_checkbox.value)
            elif op == 'Volum om x-aksen':
                handle_volume_revolution(f_expr, g_expr, a, b, axis='x')
            elif op == 'Volum om y-aksen':
                handle_volume_revolution(f_expr, g_expr, a, b, axis='y')
            elif op == 'Numerisk integral':
                # Numerisk trenger den originale strengen
                handle_numerical_integration(f_str, float(a), float(b))

        except Exception as e:
            # Vis feilmelding på en pen måte
            print(f"En feil oppstod: {e}")

run_button.on_click(run_calculation_on_click)

# Vis grensesnittet
display(input_widgets)
display(run_button)
display(output_area)

# Initialiser UI-visningen
update_ui()

# $\color{green}{\text{Kapittel 3 - Integrasjonsmetoder}}$

# $\color{blue}{\text{Kapittel 4 - Vektorer}}$

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Tab, VBox, HBox, Button, Output, FloatText, Layout, Label
from IPython.display import display

# --- MATEMATISKE HJELPEFUNKSJONER ---
def beregn_vinkel(v1, v2):
    norm_v1, norm_v2 = np.linalg.norm(v1), np.linalg.norm(v2)
    if norm_v1 == 0 or norm_v2 == 0: return 0.0
    cos_theta = np.dot(v1, v2) / (norm_v1 * norm_v2)
    return np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))

def projeksjon_vektor(v1, v2):
    dot_v2_v2 = np.dot(v2, v2)
    if dot_v2_v2 == 0: return np.zeros_like(v2)
    return (np.dot(v1, v2) / dot_v2_v2) * v2

def get_point_from_widgets(widgets):
    vals = [w.value for w in widgets]
    return np.array(vals)

# --- WIDGETS OG LAYOUT ---
layout = Layout(width='90px')
style = {'description_width': 'initial'}

# Trekant / Pyramide
analyse_widgets = {k: [FloatText(description=f"{k}{c}", layout=layout, style=style, value=0.0) for c in 'xyz']
                   for k in ['A','B','C','T']}
analyse_button = Button(description="Beregn Analyse", button_style='primary', layout=Layout(width='200px'))

# Linje og Plan
linje_plan_widgets = {k: [FloatText(description=f"{k}{c}", layout=layout, style=style, value=0.0) for c in 'xyz']
                      for k in ['P0','r','P_plan','n']}
linje_plan_button = Button(description="Beregn Skjæring", button_style='primary', layout=Layout(width='200px'))

# Basisoperasjoner
basis_widgets = {k: [FloatText(description=f"{k}{c}", layout=layout, style=style, value=0.0) for c in 'xyz']
                 for k in ['v₁','v₂']}
basis_button = Button(description="Beregn Operasjoner", button_style='primary', layout=Layout(width='200px'))

output_omrade = Output()

# --- FUNKSJONER TIL KNAPPER ---
def beregn_analyse_klikk(b):
    with output_omrade:
        output_omrade.clear_output(wait=True)
        A, B, C, T = [get_point_from_widgets(analyse_widgets[k]) for k in ['A','B','C','T']]
        try:
            AB, AC, BC = B-A, C-A, C-B
            print("--- ANALYSE AV TREKANT ABC ---")
            print(f"Sidelengder: |AB|={np.linalg.norm(AB):.3f}, |AC|={np.linalg.norm(AC):.3f}, |BC|={np.linalg.norm(BC):.3f}")
            print(f"Vinkler: ∠A={beregn_vinkel(AB, AC):.2f}°, ∠B={beregn_vinkel(-AB, BC):.2f}°, ∠C={beregn_vinkel(-AC, -BC):.2f}°")
            kryss = np.cross(AB, AC)
            print(f"Areal: {0.5*np.linalg.norm(kryss):.3f}")
            D = A + projeksjon_vektor(AC, AB)
            print(f"Fotpunkt (C→AB): {np.round(D,3)}")
            print(f"Avstand C til AB: {np.linalg.norm(C-D):.3f}")
            if np.linalg.norm(T) != 0:
                vol = abs(np.dot(kryss, T-A)) / 6.0
                print("\n--- PYRAMIDE ABCT ---")
                print(f"Volum: {vol:.3f}")
        except Exception as e:
            print(f"FEIL: {e}")

def beregn_linje_plan_klikk(b):
    with output_omrade:
        output_omrade.clear_output(wait=True)
        P0, r, P_plan, n = [get_point_from_widgets(linje_plan_widgets[k]) for k in ['P0','r','P_plan','n']]
        try:
            print("--- LINJE & PLAN ---")
            d = -np.dot(P_plan, n)
            print(f"Plan: {n[0]:.2f}x+{n[1]:.2f}y+{n[2]:.2f}z+{d:.2f}=0")
            nevner = np.dot(n, r)
            if abs(nevner)<1e-9:
                print("Linjen er parallell med planet.")
            else:
                t = np.dot(n, P_plan - P0) / nevner
                X = P0 + t*r
                vinkel = np.degrees(np.arcsin(abs(nevner)/(np.linalg.norm(r)*np.linalg.norm(n))))
                print(f"Skjæring i {np.round(X,3)}, t={t:.3f}")
                print(f"Vinkel linje ↔ plan: {vinkel:.2f}°")
        except Exception as e:
            print(f"FEIL: {e}")

def beregn_basis_klikk(b):
    with output_omrade:
        output_omrade.clear_output(wait=True)
        v1 = get_point_from_widgets(basis_widgets['v₁'])
        v2 = get_point_from_widgets(basis_widgets['v₂'])
        try:
            print("--- VEKTOR-OPERASJONER ---")
            print(f"v₁ · v₂ = {np.dot(v1,v2):.3f}")
            print(f"v₁ x v₂ = {np.round(np.cross(v1,v2),3)}")
            print(f"Vinkel = {beregn_vinkel(v1,v2):.2f}°")
            print(f"Projeksjon v₁→v₂: {np.round(projeksjon_vektor(v1,v2),3)}")
        except Exception as e:
            print(f"FEIL: {e}")

# Koble knapper
analyse_button.on_click(beregn_analyse_klikk)
linje_plan_button.on_click(beregn_linje_plan_klikk)
basis_button.on_click(beregn_basis_klikk)

# --- UI med faner ---
tab1 = VBox([
    Label("Trekant/pyramide-analysator"),
    HBox([Label("A:"), *analyse_widgets['A']]),
    HBox([Label("B:"), *analyse_widgets['B']]),
    HBox([Label("C:"), *analyse_widgets['C']]),
    HBox([Label("T (0,0,0 betyr ingen beregning):"), *analyse_widgets['T']]),
    analyse_button
])
tab2 = VBox([
    Label("Linje-plan skjæring"),
    HBox([Label("P₀:"), *linje_plan_widgets['P0']]),
    HBox([Label("Retning r:"), *linje_plan_widgets['r']]),
    HBox([Label("Punkt i plan:"), *linje_plan_widgets['P_plan']]),
    HBox([Label("Normal n:"), *linje_plan_widgets['n']]),
    linje_plan_button
])
tab3 = VBox([
    Label("Vektoroperasjoner"),
    HBox([Label("v₁:"), *basis_widgets['v₁']]),
    HBox([Label("v₂:"), *basis_widgets['v₂']]),
    basis_button
])

tabs = Tab([tab1, tab2, tab3])
tabs.set_title(0, "Trekant/Pyramide")
tabs.set_title(1, "Linje/Plan")
tabs.set_title(2, "Basisoperasjoner")

display(VBox([tabs, output_omrade]))

In [7]:
# Fullstendig analyseverktøy for kapittel 4 R2
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Tab, VBox, HBox, Button, Output, FloatText, Layout, Label, Text
from IPython.display import display
import sympy as sp
from mpl_toolkits.mplot3d import Axes3D

# --- Hjelpefunksjoner ---
def beregn_vinkel(v1, v2):
    norm_v1, norm_v2 = np.linalg.norm(v1), np.linalg.norm(v2)
    if norm_v1 == 0 or norm_v2 == 0: return 0.0
    cos_theta = np.dot(v1, v2) / (norm_v1 * norm_v2)
    return np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))

def projeksjon_vektor(v1, v2):
    dot_v2_v2 = np.dot(v2, v2)
    if dot_v2_v2 == 0: return np.zeros_like(v2)
    return (np.dot(v1, v2) / dot_v2_v2) * v2

def get_point_from_widgets(widgets):
    return np.array([w.value for w in widgets])

layout = Layout(width='90px')
style = {'description_width': 'initial'}
output_omrade = Output()

# --- Tab 1: Trekant / Pyramide ---
analyse_widgets = {k: [FloatText(description=f"{k}{c}", layout=layout, style=style, value=0.0) for c in 'xyz'] for k in 'ABCT'}
analyse_button = Button(description="Beregn Analyse", button_style='primary', layout=Layout(width='200px'))

def beregn_analyse_klikk(b):
    with output_omrade:
        output_omrade.clear_output(wait=True)
        A, B, C, T = [get_point_from_widgets(analyse_widgets[k]) for k in 'ABCT']
        try:
            AB, AC, BC = B - A, C - A, C - B
            print("--- Trekant ABC ---")
            print(f"Sider: |AB|={np.linalg.norm(AB):.3f}, |AC|={np.linalg.norm(AC):.3f}, |BC|={np.linalg.norm(BC):.3f}")
            print(f"Vinkler: ∠A={beregn_vinkel(AB, AC):.2f}°, ∠B={beregn_vinkel(-AB, BC):.2f}°, ∠C={beregn_vinkel(-AC, -BC):.2f}°")
            areal = 0.5 * np.linalg.norm(np.cross(AB, AC))
            print(f"Areal: {areal:.3f}")
            D = A + projeksjon_vektor(AC, AB)
            print(f"Fotpunkt (C→AB): {np.round(D,3)}, Avstand: {np.linalg.norm(C - D):.3f}")
            if np.linalg.norm(T) != 0:
                volum = abs(np.dot(np.cross(AB, AC), T - A)) / 6.0
                print("--- Pyramide ABCT ---")
                print(f"Volum: {volum:.3f}")
        except Exception as e:
            print(f"FEIL: {e}")

analyse_button.on_click(beregn_analyse_klikk)

tab1 = VBox([
    Label("Trekant og pyramideanalyse"),
    *[HBox([Label(f"{k}:")] + analyse_widgets[k]) for k in 'ABCT'],
    analyse_button
])

# --- Tab 2: Linje og Plan ---
linje_plan_widgets = {k: [FloatText(description=f"{k}{c}", layout=layout, style=style, value=0.0) for c in 'xyz'] for k in ['P0','r','P_plan','n']}
linje_plan_button = Button(description="Beregn Skjæring", button_style='primary', layout=Layout(width='200px'))

def beregn_linje_plan_klikk(b):
    with output_omrade:
        output_omrade.clear_output(wait=True)
        P0, r, P_plan, n = [get_point_from_widgets(linje_plan_widgets[k]) for k in ['P0','r','P_plan','n']]
        try:
            print("--- Linje og Plan ---")
            d = -np.dot(P_plan, n)
            print(f"Planlikning: {n[0]:.2f}x+{n[1]:.2f}y+{n[2]:.2f}z+{d:.2f}=0")
            nevner = np.dot(n, r)
            if abs(nevner)<1e-9:
                print("Linjen er parallell med planet.")
            else:
                t = np.dot(n, P_plan - P0) / nevner
                X = P0 + t*r
                print(f"Skjæringspunkt: {np.round(X,3)} ved t={t:.3f}")
                vinkel = np.degrees(np.arcsin(abs(nevner)/(np.linalg.norm(r)*np.linalg.norm(n))))
                print(f"Vinkel linje ↔ plan: {vinkel:.2f}°")
        except Exception as e:
            print(f"FEIL: {e}")

linje_plan_button.on_click(beregn_linje_plan_klikk)

tab2 = VBox([
    Label("Linje og plan: skjæring og vinkel"),
    *[HBox([Label(f"{k}:")] + linje_plan_widgets[k]) for k in ['P0','r','P_plan','n']],
    linje_plan_button
])

# --- Tab 3: Vektoroperasjoner ---
basis_widgets = {k: [FloatText(description=f"{k}{c}", layout=layout, style=style, value=0.0) for c in 'xyz'] for k in ['v1','v2']}
basis_button = Button(description="Beregn Operasjoner", button_style='primary', layout=Layout(width='200px'))

def beregn_basis_klikk(b):
    with output_omrade:
        output_omrade.clear_output(wait=True)
        v1 = get_point_from_widgets(basis_widgets['v1'])
        v2 = get_point_from_widgets(basis_widgets['v2'])
        try:
            print("--- Vektoroperasjoner ---")
            print(f"v1 · v2 = {np.dot(v1, v2):.3f}")
            print(f"v1 x v2 = {np.round(np.cross(v1, v2), 3)}")
            print(f"Vinkel mellom v1 og v2: {beregn_vinkel(v1, v2):.2f}°")
            print(f"Projeksjon av v1 på v2: {np.round(projeksjon_vektor(v1, v2),3)}")
        except Exception as e:
            print(f"FEIL: {e}")

basis_button.on_click(beregn_basis_klikk)

tab3 = VBox([
    Label("Vektoroperasjoner"),
    HBox([Label("v1:")] + basis_widgets['v1']),
    HBox([Label("v2:")] + basis_widgets['v2']),
    basis_button
])

# --- Tab 4: Symbolsk ortogonalitet ---
symb_widgets = [Text(description="v1", style=style), Text(description="v2", style=style)]
symb_button = Button(description="Finn x slik at v1 ⊥ v2", button_style='info', layout=Layout(width='200px'))

def beregn_symbolsk_ortogonalitet(b):
    with output_omrade:
        output_omrade.clear_output(wait=True)
        try:
            v1_expr = sp.sympify(symb_widgets[0].value)
            v2_expr = sp.sympify(symb_widgets[1].value)
            v1_vec = sp.Matrix(v1_expr)
            v2_vec = sp.Matrix(v2_expr)
            x = list(v1_vec.free_symbols.union(v2_vec.free_symbols))
            dot = v1_vec.dot(v2_vec)
            print(f"Skalarprodukt: {dot}")
            if x:
                løsninger = sp.solve(dot, *x)
                print(f"Løsning(er) for ortogonalitet: {løsninger}")
            else:
                print("Ingen variabel å løse for.")
        except Exception as e:
            print(f"FEIL: {e}")

symb_button.on_click(beregn_symbolsk_ortogonalitet)

tab4 = VBox([
    Label("Finn x slik at to vektorer er ortogonale"),
    *symb_widgets,
    symb_button
])

# --- Tab 5: Plan gjennom 3 punkt ---
plan3_widgets = {k: [FloatText(description=f"{k}{c}", layout=layout, style=style) for c in 'xyz'] for k in ['P','Q','R']}
plan3_button = Button(description="Finn plan", button_style='success', layout=Layout(width='200px'))

def finn_plan_3punkt(b):
    with output_omrade:
        output_omrade.clear_output(wait=True)
        try:
            P = get_point_from_widgets(plan3_widgets['P'])
            Q = get_point_from_widgets(plan3_widgets['Q'])
            R = get_point_from_widgets(plan3_widgets['R'])
            n = np.cross(Q - P, R - P)
            d = -np.dot(n, P)
            print(f"Plan gjennom P, Q, R:\n{n[0]:.2f}x + {n[1]:.2f}y + {n[2]:.2f}z + {d:.2f} = 0")
        except Exception as e:
            print(f"FEIL: {e}")

plan3_button.on_click(finn_plan_3punkt)

tab5 = VBox([
    Label("Finn planlikning gjennom 3 punkt"),
    HBox([Label("P:"), *plan3_widgets['P']]),
    HBox([Label("Q:"), *plan3_widgets['Q']]),
    HBox([Label("R:"), *plan3_widgets['R']]),
    plan3_button
])

# --- Tab 6: Visualisering ---
vis_widgets = {k: [FloatText(description=f"{k}{c}", layout=layout, style=style) for c in 'xyz'] for k in ['A','B','C']}
vis_button = Button(description="Vis i 3D", button_style='warning', layout=Layout(width='150px'))

def vis_3d_plot(b):
    with output_omrade:
        output_omrade.clear_output(wait=True)
        try:
            A = get_point_from_widgets(vis_widgets['A'])
            B = get_point_from_widgets(vis_widgets['B'])
            C = get_point_from_widgets(vis_widgets['C'])
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
            ax.plot([A[0], B[0]], [A[1], B[1]], [A[2], B[2]], label='AB', color='blue')
            ax.plot([A[0], C[0]], [A[1], C[1]], [A[2], C[2]], label='AC', color='green')
            ax.plot([B[0], C[0]], [B[1], C[1]], [B[2], C[2]], label='BC', color='red')
            ax.scatter(*A, color='blue', label='A')
            ax.scatter(*B, color='green', label='B')
            ax.scatter(*C, color='red', label='C')
            ax.set_xlabel('X')
            ax.set_ylabel('Y')
            ax.set_zlabel('Z')
            ax.legend()
            plt.show()
        except Exception as e:
            print(f"FEIL: {e}")

vis_button.on_click(vis_3d_plot)

tab6 = VBox([
    Label("Visualiser trekant i 3D (A, B, C)"),
    HBox([Label("A:"), *vis_widgets['A']]),
    HBox([Label("B:"), *vis_widgets['B']]),
    HBox([Label("C:"), *vis_widgets['C']]),
    vis_button
])

# --- Tab 7: CAS (Volum og avstand fra punkt til plan) ---
cas_widgets = {k: [FloatText(description=f"{k}{c}", layout=layout, style=style) for c in 'xyz'] for k in ['A','B','C','D']}
cas_button = Button(description="Beregn CAS", button_style='danger', layout=Layout(width='200px'))

def beregn_volum_avstand(b):
    with output_omrade:
        output_omrade.clear_output(wait=True)
        try:
            A = get_point_from_widgets(cas_widgets['A'])
            B = get_point_from_widgets(cas_widgets['B'])
            C = get_point_from_widgets(cas_widgets['C'])
            D = get_point_from_widgets(cas_widgets['D'])
            AB = B - A
            AC = C - A
            AD = D - A
            kryss = np.cross(AB, AC)
            volum = abs(np.dot(kryss, AD)) / 6.0
            print("--- CAS-beregninger ---")
            print(f"Volum av pyramiden ABCD: {volum:.3f}")
            n = np.cross(AB, AC)
            avstand = abs(np.dot(n, D - A)) / np.linalg.norm(n)
            print(f"Avstand fra D til grunnflaten ABC: {avstand:.3f}")
        except Exception as e:
            print(f"FEIL: {e}")

cas_button.on_click(beregn_volum_avstand)

tab7 = VBox([
    Label("Volum og avstand fra punkt til plan (CAS)"),
    HBox([Label("A:"), *cas_widgets['A']]),
    HBox([Label("B:"), *cas_widgets['B']]),
    HBox([Label("C:"), *cas_widgets['C']]),
    HBox([Label("D:"), *cas_widgets['D']]),
    cas_button
])

# --- Tab 8: Punkt på linje (t-test) ---
punktlinje_widgets = {k: [FloatText(description=f"{k}{c}", layout=layout, style=style) for c in 'xyz'] for k in ['P','Q','M']}
punktlinje_button = Button(description="Sjekk om M ligger på PQ", button_style='info', layout=Layout(width='220px'))

def test_om_punkt_ligger_på_linje(b):
    with output_omrade:
        output_omrade.clear_output(wait=True)
        try:
            P = get_point_from_widgets(punktlinje_widgets['P'])
            Q = get_point_from_widgets(punktlinje_widgets['Q'])
            M = get_point_from_widgets(punktlinje_widgets['M'])
            PQ = Q - P
            PM = M - P
            forhold = [PM[i]/PQ[i] if PQ[i] != 0 else None for i in range(3)]
            if all(p == forhold[0] or p is None for p in forhold):
                print(f"✅ Punktet M ligger på linjen gjennom P og Q (t = {forhold[0]:.3f})")
            else:
                print("❌ Punktet M ligger ikke på linjen gjennom P og Q")
        except Exception as e:
            print(f"FEIL: {e}")

punktlinje_button.on_click(test_om_punkt_ligger_på_linje)

tab8 = VBox([
    Label("Sjekk om et punkt M ligger på linjen gjennom P og Q"),
    HBox([Label("P:"), *punktlinje_widgets['P']]),
    HBox([Label("Q:"), *punktlinje_widgets['Q']]),
    HBox([Label("M:"), *punktlinje_widgets['M']]),
    punktlinje_button
])

# --- Tab 9: Symbolsk løsning med flere ukjente ---
symb_multi_widgets = [Text(description="v1", style=style), Text(description="v2", style=style)]
symb_multi_button = Button(description="Løs for ukjente", button_style='success', layout=Layout(width='160px'))

def løs_symbolsk_flere_ukjente(b):
    with output_omrade:
        output_omrade.clear_output(wait=True)
        try:
            v1_expr = sp.sympify(symb_multi_widgets[0].value)
            v2_expr = sp.sympify(symb_multi_widgets[1].value)
            v1_vec = sp.Matrix(v1_expr)
            v2_vec = sp.Matrix(v2_expr)
            symboler = list(v1_vec.free_symbols.union(v2_vec.free_symbols))
            dot = v1_vec.dot(v2_vec)
            print(f"Skalarprodukt: {dot}")
            løsninger = sp.solve(dot, *symboler, dict=True)
            print(f"Løsninger: {løsninger}")
        except Exception as e:
            print(f"FEIL: {e}")

symb_multi_button.on_click(løs_symbolsk_flere_ukjente)

tab9 = VBox([
    Label("Symbolsk løsning med flere ukjente (f.eks. x og t)"),
    *symb_multi_widgets,
    symb_multi_button
])

# --- Samle alle faner ---
tabs = Tab(children=[tab1, tab2, tab3, tab4, tab5, tab6, tab7, tab8, tab9])
tabs.set_title(0, "Trekant/Pyramide")
tabs.set_title(1, "Linje/Plan")
tabs.set_title(2, "Vektorops")
tabs.set_title(3, "Ortogonalitet")
tabs.set_title(4, "Plan 3 punkt")
tabs.set_title(5, "3D-visning")
tabs.set_title(6, "CAS: Volum/Avstand")
tabs.set_title(7, "Punkt på linje")
tabs.set_title(8, "Symbolsk flere ukjente")

display(VBox([tabs, output_omrade]))

# $\color{lightblue}{\text{Kapittel 5 - Trigonometri}}$

In [4]:
print(5+5)

10


# $\color{coral}{\text{Kapittel 6 - Funksjoner og kurver}}$